## Extraction

i

In [2]:
file1 = open("output.txt","r")
body = f"""{file1.read()}"""
body

'extracted successfully\ntransformed successfully\nAn error occurred: (psycopg2.OperationalError) could not translate host name "testtech.postgres.database.azure.com" to address: Name or service not known\n\n(Background on this error at: https://sqlalche.me/e/20/e3q8)\n'

## Transformation

In [27]:
import pandas as pd
from forex_python.converter import CurrencyRates

def transform(data):
    df = pd.DataFrame(data)
    df = df.iloc[:,list(range(9)) + [-1]]
    df = df.drop('image',axis=1)


    def convert_price(row, to_currency):
        c = CurrencyRates()
        price = row['current_price']
        from_currency = 'USD'
        exchange_rate = c.get_rate(from_currency,to_currency)
        converted_price = price * exchange_rate

        return converted_price

    currencies = ['EUR', 'GBP', 'JPY']

    for currency in currencies:
        df[f'price_in_{currency}'] = df.apply(lambda row: convert_price(row, currency), axis = 1)
    print('transformed successfully')
    return df

## Loading

In [28]:
import psycopg2 #pip install psycopg2 sqlalchemy 
from sqlalchemy import create_engine
def loading(dataframe):
    try:
        
        # Create a dictionary of database connection parameters
        connection_params = {
            "host": "testtech.postgres.database.azure.com",
            "port": "5432",
            "user": "testtech",
            "password": "Your_password",
            "database": "postgres"
        }

        # Create a SQLAlchemy engine for connecting to the database
        engine = create_engine(
            f'postgresql+psycopg2://{connection_params["user"]}:{connection_params["password"]}@{connection_params["host"]}:{connection_params["port"]}/{connection_params["database"]}'
        )

        # Append the DataFrame contents to the existing table
        df.to_sql("trending_coins", engine, if_exists='append', index=False)

        print('Database successfully updated')

    except Exception as e:
        print("An error occurred:", e)
        # Optionally: Log the error for further analysis

    finally:
        # Close the database connection gracefully
        if engine:
            engine.dispose()

In [29]:
data = extract()
dataframe = transform(data)
loading(dataframe)

extracted successfully
transformed successfully
Database successfully updated
